In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [2]:
df = pd.read_csv('data/secondary-prediction/secondary_prediction_players_statistics.csv')
df.groupby(['game']).size().sort_values(ascending=True)

game
ESPORTSTMNT01_2690210    2
ESPORTSTMNT03_2564859    2
ESPORTSTMNT03_2565045    2
ESPORTSTMNT03_2565333    2
ESPORTSTMNT03_2565350    2
                        ..
ESPORTSTMNT01_2781985    2
ESPORTSTMNT01_2781996    2
ESPORTSTMNT01_2781997    2
ESPORTSTMNT01_2781781    2
NA1_4217912820           2
Length: 2621, dtype: int64

In [3]:
y = df['firstTower'].copy()
X = df.drop(['game', 'firstBlood', 'kills', 'deaths', 'firstTower', 'firstHerald', 'dragons', 'barons', 'inhibitors', 'towers', 'heralds'], axis=1)
X

,flagSide,topGP,topWR,topKDA,jungleGP,jungleWR,jungleKDA,midGP,midWR,midKDA,carryGP,carryWR,carryKDA,suppGP,suppWR,suppKDA
0,1,3,0.67,4.0,0,0.00,0.0,12,0.42,4.2,1,0.00,4.0,1,1.00,9.0
1,0,6,0.50,5.1,4,0.50,4.7,0,0.00,0.0,1,0.00,1.3,11,0.36,2.4
2,1,4,0.25,2.2,2,0.00,1.2,3,0.33,4.1,0,0.00,0.0,0,0.00,0.0
3,0,4,0.00,1.5,3,0.33,1.1,2,0.50,4.2,4,0.50,3.0,0,0.00,0.0
4,1,3,0.00,1.6,0,0.00,0.0,1,0.00,0.7,5,0.40,3.1,0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237,0,9,0.67,2.3,7,0.57,5.3,27,0.48,5.1,22,0.50,4.9,16,0.31,2.1
5238,1,0,0.00,0.0,5,0.40,4.2,7,0.71,7.1,2,0.00,1.4,7,0.00,1.0
5239,0,3,0.33,1.8,12,0.67,6.7,4,0.75,3.3,9,0.67,3.8,7,0.71,3.4
5240,1,3,1.00,6.0,7,0.43,3.7,12,0.67,4.2,8,0.88,9.5,0,0.00,0.0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      flagSide  topGP  topWR  topKDA  jungleGP  jungleWR  jungleKDA  midGP  \
2709         0      5   0.20     1.5         1      1.00        5.0      8   
4552         1      0   0.00     0.0         0      0.00        0.0      4   
1315         0      2   0.00     0.4         6      0.00        1.9      8   
2668         1      7   0.00     1.4        15      0.40        4.0      8   
2398         1      2   1.00    15.0         4      0.75       10.2      5   
...        ...    ...    ...     ...       ...       ...        ...    ...   
3752         1     24   0.50     2.5        27      0.48        3.1      8   
3930         1      0   0.00     0.0         3      0.67        6.3      1   
3393         0     11   0.55     3.4         5      0.40        2.5     10   
4462         1      3   0.67     2.6         5      1.00       11.9      0   
4744         1      0   0.00     0.0         4      0.75        3.8      0   

      midWR  midKDA  carryGP  carryWR  carryKDA  suppGP  suppWR

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


C:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.533842,0.534381,0.519084,0.526621
1,Support Vector Machine (Linear Kernel),0.529075,0.529528,0.513359,0.521318
2,Support Vector Machine (RBF Kernel),0.568160,0.568401,0.562977,0.565676
3,Decission Tree,0.522402,0.520211,0.564885,0.541629
4,Adaboost,0.559581,0.561265,0.541985,0.551456
5,Random Forest,0.551954,0.549632,0.570611,0.559925
6,Gradient Boosting Classifier,0.581506,0.579143,0.593511,0.586239
